In [1]:
import selenium
from selenium import webdriver
import pandas as pd 
import warnings 
warnings.filterwarnings('ignore')
from selenium.common.exceptions import NoSuchElementException
import time

**Q1. Write a python program which searches all the product under a particular product from www.amazon.in.
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search
for guitars.**

In [118]:
#connecting  and giving url to the webdriver 
driver=webdriver.Chrome('chromedriver.exe')
url=('https://www.amazon.in/')
driver.get(url)

# obtaining the search option on our web driver
search = driver.find_element_by_id('twotabsearchtextbox')

# obtaining the item to be searched from the user
search = driver.find_element_by_id("twotabsearchtextbox")
item = str(input("Enter item name that you want to search: "))
search.send_keys(item)

# performing a click using xpath function in our web driver
search_btn = driver.find_element_by_xpath("//span[@class='nav-search-submit-text nav-sprite nav-progressive-attribute']")
search_btn.click()
time.sleep(5)

Enter item name that you want to search: Football


**Q2)In the above question, now scrape the following details of each product listed in first 3 pages of your
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search
results then scrape all the products available under that product name. Details to be scraped are: "Brand
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.**

In [119]:
URL=[]  #Empty list
for page in range(0,3):
    for i in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-medium']//h2"):
        URL.append(i.find_element_by_xpath(".//a").get_attribute('href'))
    try:
        next_page=driver.find_element_by_xpath("//div[@class='a-text-center']/ul/li[@class='a-last']/a").get_attribute('href')
        driver.get(next_page)
        driver.refresh()   
    except:
        break   

In [120]:
#Taking the empty lists
brand_name=[]
product_name=[]
Rating=[]
no_of_ratings=[]
price=[]
returns=[]
delivery=[]
availability=[]
other_details=[]
product_url=[]
     

In [121]:
for i in URL:
    driver.get(i)
    product_url.append(i)
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the brand name
    try:
        brand_name.append(driver.find_element_by_id("productTitle").text.split(' ')[0])
    except NoSuchElementException as e:
        brand_name.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds   
    
    #Extracting the product name
    try:
        string=' '  #Empty string
        for j in driver.find_element_by_id("productTitle").text.split(' ')[1:]:
            string=string+' '+j  #Extracting the text and appending to empty string
        product_name.append(string)  #Appending the string having the extracted text
    except NoSuchElementException as e:
            product_name.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds     
    
    #Extracting the ratings
    try:
        Rating.append(driver.find_element_by_id("acrPopover").get_attribute('title')) 
    except NoSuchElementException as e:
        Rating.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the number of ratings
    try:
        no_of_ratings.append(driver.find_element_by_id("acrCustomerReviewText").text) 
    except NoSuchElementException as e:
        no_of_ratings.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the price
    try:
        try:
            price.append(driver.find_element_by_id("priceblock_saleprice").text) 
        except:
            try:
                price.append(driver.find_element_by_id("priceblock_dealprice").text)
            except:
                price.append(driver.find_element_by_id("priceblock_ourprice").text)
    except NoSuchElementException as e:
        price.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the Return/Exchange
    try:
        returns.append(driver.find_element_by_xpath("//div[@id='RETURNS_POLICY']").text)
    except NoSuchElementException as e:
        returns.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting Expected Delivery
    try:
        delivery.append(driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b').text)
    except NoSuchElementException as e:
        delivery.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds  
    
    #Extracting the availability of headphones
    try:
        availability.append(driver.find_element_by_xpath("//div[@id='availability']").text)
    except NoSuchElementException as e:    
        availability.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds  
    
    #Extracting the other details available
    try:
        details=[i.text.replace('\n','---') for i in driver.find_element_by_id("productDetails_techSpec_section_1").find_elements_by_xpath(".//tbody")]
        other_details.append(details[0])
    except NoSuchElementException as e:     
        other_details.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds

In [122]:
#Checking the length of data extracted
print(len(brand_name),len(product_name),len(Rating),len(no_of_ratings),len(price),len(returns),len(delivery),
      len(availability),len(other_details),len(product_url))

63 63 63 63 63 63 63 63 63 63


In [123]:
#Creating a dataframe and checking the data extracted
amazon=pd.DataFrame({'Brand Name':brand_name,'Name of Product':product_name,'Rating':Rating,'No of Ratings':no_of_ratings,
                     'Price':price,'Return/Exchange':returns,'Expected Delivery':delivery,'Availability':availability,
                     'Other details':other_details,'Product URL':product_url})
amazon

,Brand Name,Name of Product,Rating,No of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Other details,Product URL
0,VOODANIA,Telstar Combo Pack 1 Football & 1 pump PU ma...,3.6 out of 5 stars,"2,186 ratings",-,10 Days Replacement,"Monday, Jan 31",In stock.,Age Range Adult---Batteries Included No---Bran...,https://www.amazon.in/gp/slredirect/picassoRed...
1,RADION,"City Football Size 5, Diameter: 26 cm (Pack ...",-,-,-,10 Days Returnable,"Monday, Jan 31",Only 1 left in stock.,Brand RADION---Included Components 1 CITY Foot...,https://www.amazon.in/gp/slredirect/picassoRed...
2,RADION,"City Football Size 5, Diameter: 26 cm (Pack ...",-,-,-,10 Days Replacement,"Monday, Jan 31",Only 1 left in stock.,Brand RADION---Included Components 1 CITY Foot...,https://www.amazon.in/gp/slredirect/picassoRed...
3,RADION,"FIFA Bluestar Football Size 5, Diameter: 26 ...",-,-,-,10 Days Returnable,"Monday, Jan 31",In stock.,Brand RADION---Included Components 1 American ...,https://www.amazon.in/gp/slredirect/picassoRed...
4,Nivia,Storm Football - Size 5,4.0 out of 5 stars,"27,555 ratings",-,10 Days Replacement,"Monday, Jan 31",In stock.,Age Range Adult---Batteries Included No---Bran...,https://www.amazon.in/Nivia-Storm-Football-Siz...
...,...,...,...,...,...,...,...,...,...,...
58,NIVIA,Snow Storm Machine Stitched Football,3.8 out of 5 stars,14 ratings,-,10 Days Replacement,Jan 30 - 31,Available to ship in 1-2 days.,Brand Nivia---Color Blue---Material Type Other...,https://www.amazon.in/NIVIA-Storm-Machine-Stit...
59,Avatoz,Polyurethane Football PARLODA for All Age Gr...,3.6 out of 5 stars,91 ratings,-,10 Days Replacement,"Monday, Jan 31",In stock.,"Brand Avatoz---Included Components 1 PUMP, 1 F...",https://www.amazon.in/gp/slredirect/picassoRed...
60,VOODANIA,Premium Brazuca Polyurethane Football With P...,3.4 out of 5 stars,112 ratings,-,10 Days Replacement,"Monday, Jan 31",In stock.,Age Range Adult---Batteries Included No---Bran...,https://www.amazon.in/gp/slredirect/picassoRed...
61,Avatoz,felota 1 Football 1 Pump for All Round Perfo...,3.6 out of 5 stars,13 ratings,-,10 Days Returnable,"Monday, Jan 31",In stock.,Brand Avatoz---Included Components FOOTBALL AN...,https://www.amazon.in/gp/slredirect/picassoRed...


In [127]:
amazon.to_csv(r'C:\Users\Mehul\Desktop\internship.csv',index=False)

**Q3)Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.**

In [33]:
#connecting and giving url to the web driver 
driver=webdriver.Chrome('chromedriver.exe')
driver.get('https://images.google.com/')

#finding web element on search bar 
time.sleep(3)
search_bar=driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_bar.send_keys('fruits')

#clicking on search button
time.sleep(3)
search_btn=driver.find_element_by_xpath('//button[@class="Tg7LZd"]')
search_btn.click()

#scarping fruits image
fruits_img=[]
fruits_url=driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
for i in fruits_url:
    fruits_img.append(i.get_attribute('src'))

In [18]:
#connecting and giving url to the web driver 
driver=webdriver.Chrome('chromedriver.exe')
driver.get('https://images.google.com/')

#finding web element on search bar 
time.sleep(3)
search_bar=driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_bar.send_keys('cars')

#clicking on search button
time.sleep(3)
search_btn=driver.find_element_by_xpath('//button[@class="Tg7LZd"]')
search_btn.click()

#scraping cars images
cars_img=[]
cars_url=driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
for i in cars_url:
    cars_img.append(i.get_attribute('src'))

In [21]:
#connecting and giving url to the web driver 
driver=webdriver.Chrome('chromedriver.exe')
driver.get('https://images.google.com/')

#finding web element on search bar 
time.sleep(3)
search_bar=driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_bar.send_keys('Machine Learning')

#clicking on search button
time.sleep(3)
search_btn=driver.find_element_by_xpath('//button[@class="Tg7LZd"]')
search_btn.click()

#scraping Machine learning images
ML_img=[]
ML_url=driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
for i in ML_url:
    ML_img.append(i.get_attribute('src'))

In [24]:
#connecting and giving url to the web driver 
driver=webdriver.Chrome('chromedriver.exe')
driver.get('https://images.google.com/')

#finding web element on search bar 
time.sleep(3)
search_bar=driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_bar.send_keys('guitar')

#clicking on search button
time.sleep(3)
search_btn=driver.find_element_by_xpath('//button[@class="Tg7LZd"]')
search_btn.click()

#scraping guitar images
guitar_img=[]
guitar_url=driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
for i in guitar_url:
    guitar_img.append(i.get_attribute('src'))

In [27]:
#connecting and giving url to the web driver 
driver=webdriver.Chrome('chromedriver.exe')
driver.get('https://images.google.com/')

#finding web element on search bar 
time.sleep(3)
search_bar=driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_bar.send_keys('cakes')

#clicking on search button
time.sleep(3)
search_btn=driver.find_element_by_xpath('//button[@class="Tg7LZd"]')
search_btn.click()

#scarping cakes image 
cakes_img=[]
cakes_url=driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
for i in cakes_url:
    cakes_img.append(i.get_attribute('src'))

In [34]:
#creating the dataframe
df=pd.DataFrame({})
df['fruits']=fruits_img[:10]
df['cars']=cars_img[:10]
df['Machine Learning']=ML_img[:10]
df['guitar']=guitar_img[:10]
df['cakes']=cakes_img[:10]
df

,fruits,cars,Machine Learning,guitar,cakes
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
6,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


**Q4)Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the
details is missing then replace it by “- “. Save your results in a dataframe and CSV**

In [56]:
#connecting and giving url to the web driver 
driver=webdriver.Chrome("chromedriver.exe")
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(3)  # waiting for 3 seconds here
click_btn = driver.find_element_by_xpath("//div[@class='_2QfC02']//button").click()

#giving inputs to search bar
search_bar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']//input")
search_bar.send_keys("Smartphones")
search_btn = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
search_btn.click()
time.sleep(5)

In [57]:
# fetching urls of every smartphone on Flipkart
urls = []
for i in driver.find_elements_by_xpath("//a[@class='_1fQZEK']"):
    urls.append(i.get_attribute("href"))

# getting the required data inside the empty lists
brand_names = []
name = []
color = []
RAM = []
storage = []
prim_cam = []
sec_cam = []
Display_size = []
display_resolution = []
processor = []
processor_core = []
battery_cap = []
prices = []
product_url = []

# fetching battery capacity
for i in driver.find_elements_by_xpath("//div[@class='fMghEO']"):
    try:
        bat_cap = driver.find_element_by_xpath("//ul[@class='_1xgFaf']//li[4]")
        battery_cap.append(bat_cap.text)
    except NoSuchElementException:
        battery_cap.append('-')
for i in urls:
    driver.get(i)
    time.sleep(3)
    
    # fetching brand names
    try:
        br_name = driver.find_element_by_xpath("/html/body/div/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/div[4]/a")
        brand_names.append(br_name.text.replace('Mobiles',''))
    except NoSuchElementException:
        brand_names.append('-')
    # fetching the smartphone names
    try:
        smt_name = driver.find_element_by_xpath("//h1[@class='yhB1nd']//span")
        name.append(smt_name.text)
    except NoSuchElementException:
        name.append('-')
    
    # fetching colors of smartphone
    try:
        clr = driver.find_element_by_xpath("//table[@class='_14cfVK']//tr[4]//td[2]")
        color.append(clr.text)
    except NoSuchElementException:
        color.append('-')
    time.sleep(2)
    
    # getting the read more button
    try:
        read_more_btn = driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
    except NoSuchElementException:
        pass
    time.sleep(3)
    
    # fetching the display size
    try:
        disp_size = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[2]//tr[1]//td[2]")
        Display_size.append(disp_size.text)
    except NoSuchElementException:
        Display_size.append
        
    # fetching the display resolution
    try:
        disp_res = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[2]//tr[2]//td[2]")
        display_resolution.append(disp_res.text)
    except NoSuchElementException:
        display_resolution.append('-')
        
    # fetching the processor information
    try:
        pro = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[3]//tr[2]//td[2]")
        processor.append(pro.text)
    except NoSuchElementException:
        processor.append('-')
        
    # fetching the processor and number of core details
    try:
        pro_core = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[3]//tr[3]//td[2]")
        processor_core.append(pro_core.text)
    except NoSuchElementException:
        processor_core.append('-')
        
    # fetching the storage/ROM details
    try:
        rom = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[4]//tr[1]//td[2]")
        storage.append(rom.text)
    except NoSuchElementException:
        storage.append('-')
        
    # fetching the RAM information
    try:
        ram = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[4]//tr[2]//td[2]")
        RAM.append(ram.text)
    except NoSuchElementException:
        RAM.append('-')
    time.sleep(2)
    
    # fetching the primary camera information
    try:
        p_cam = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[5]//tr[2]//td[2]")
        prim_cam.append(p_cam.text)
    except NoSuchElementException:
        prim_cam.append('-')
        
    # fetching the secondary camera information
    try:
        s_cam = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[5]//tr[5]//td[2]")
        sec_cam.append(s_cam.text)
    except NoSuchElementException:
        sec_cam.append('-')        
   
    # fetching the price of the smartphone
    try:
        price = driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")
        prices.append(price.text.replace('₹','Rs. '))
    except NoSuchElementException:
        prices.append('-')
    
    
# creating a dataframe now with the obtainined details
data = list(zip(brand_names,name,color,RAM,storage,prim_cam,sec_cam,Display_size,display_resolution,processor,
                processor_core,battery_cap,prices,urls))
df = pd.DataFrame(data, columns = ["Brand Name","Smartphone Name","Color","RAM","Storage/ROM","Primary Camera",
                                   "Secondary Camera","Display Size","Display Resolution","Processor","Processor-Core",
                                   "Battery Capacity","Price","Product Url"])
df

,Brand Name,Smartphone Name,Color,RAM,Storage/ROM,Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor-Core,Battery Capacity,Price,Product Url
0,realme,"realme Narzo 50i (Mint Green, 32 GB) (2 GB RAM)",Mint Green,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,SC9863A,Octa Core,5000 mAh Battery,"Rs. 7,499",https://www.flipkart.com/realme-narzo-50i-mint...
1,realme,"realme Narzo 50i (Carbon Black, 32 GB) (2 GB ...",Carbon Black,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,SC9863A,Octa Core,5000 mAh Battery,"Rs. 7,499",https://www.flipkart.com/realme-narzo-50i-carb...
2,realme,"realme Narzo 50i (Mint Green, 64 GB) (4 GB RAM)",Mint Green,4 GB,64 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,SC9863A,Octa Core,5000 mAh Battery,"Rs. 8,999",https://www.flipkart.com/realme-narzo-50i-mint...
3,LAVA,"LAVA Z3 (Striped Cyan, 32 GB) (3 GB RAM)",Striped Cyan,3 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio A20,Quad Core,5000 mAh Battery,"Rs. 7,499",https://www.flipkart.com/lava-z3-striped-cyan-...
4,realme,"realme Narzo 50A (Oxygen Green, 64 GB) (4 GB ...",Oxygen Green,4 GB,64 GB,50MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G85,Octa Core,5000 mAh Battery,"Rs. 11,699",https://www.flipkart.com/realme-narzo-50a-oxyg...
5,realme,"realme Narzo 50A (Oxygen Blue, 128 GB) (4 GB ...",Oxygen Blue,4 GB,128 GB,50MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G85,Octa Core,5000 mAh Battery,"Rs. 12,499",https://www.flipkart.com/realme-narzo-50a-oxyg...
6,realme,"realme Narzo 50A (Oxygen Green, 128 GB) (4 GB...",Oxygen Green,4 GB,128 GB,50MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G85,Octa Core,5000 mAh Battery,"Rs. 12,499",https://www.flipkart.com/realme-narzo-50a-oxyg...
7,SAMSUNG,"SAMSUNG Galaxy F12 (Sea Green, 64 GB) (4 GB RAM)",Sea Green,4 GB,64 GB,48MP + 5MP + 2MP + 2MP,8MP Front Camera,16.55 cm (6.515 inch),1600 x 720 Pixels,Exynos 850,Octa Core,5000 mAh Battery,"Rs. 11,499",https://www.flipkart.com/samsung-galaxy-f12-se...
8,realme,"realme Narzo 50i (Carbon Black, 64 GB) (4 GB ...",Carbon Black,4 GB,64 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,SC9863A,Octa Core,5000 mAh Battery,"Rs. 8,999",https://www.flipkart.com/realme-narzo-50i-carb...
9,realme,"realme Narzo 50A (Oxygen Blue, 64 GB) (4 GB RAM)",Oxygen Blue,4 GB,64 GB,50MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G85,Octa Core,5000 mAh Battery,"Rs. 11,699",https://www.flipkart.com/realme-narzo-50a-oxyg...


**Q5)Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps**

In [21]:
#connecting and giving url to the web driver 
driver = webdriver.Chrome("chromedriver.exe")
url="https://www.google.co.in/maps"
driver.get(url)
srch_box = driver.find_element_by_class_name("tactile-searchbox-input")  # getting the search bar
place = str(input("Enter the place name which you want to search on Google Maps: "))  # taking input from user to search for location
srch_box.send_keys(place)    

Enter the place name which you want to search on Google Maps: delhi


In [23]:
# clicking on the search button
submit = driver.find_element_by_id("searchbox-searchbutton")
time.sleep(5)
submit.click()

In [24]:
current_url = driver.current_url # coordinates are presents in the URL so fetching current ur
import re

try:
    lat = re.findall('(\d+(\.\d*)?)', current_url)[0]   # fetching the latitude data
    lon = re.findall('(\d+(\.\d*)?)', current_url)[1]  # fetching the longitude data
    print("Geospatial Coordinates for", place + " on Google Maps are: ")
    print("Latitude  :", lat[0])
    print("Longitude :", lon[0])
    
except Exception as e:
    print("Error: ", str(e))

Geospatial Coordinates for delhi on Google Maps are: 
Latitude  : 28.5966018
Longitude : 77.297967


**Q6)Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21)
from trak.in**


In [49]:
#connecting and giving url to the web driver  
driver = webdriver.Chrome('chromedriver.exe')
url = 'https://trak.in/india-startup-funding-investment-2015/'
driver.get(url)

In [50]:
dt=[]
name=[]
ind=[]
sv=[]
city=[]
inv=[]
invt=[]
amount=[]

# scraping the July month data
d= driver.find_elements_by_xpath("//table[@id='tablepress-48']//td[@class='column-2']")
for i in d:
    dt.append(i.text)    
n= driver.find_elements_by_xpath("//table[@id='tablepress-48']//td[@class='column-3']")
for i in n:
    name.append(i.text)    
x= driver.find_elements_by_xpath("//table[@id='tablepress-48']//td[@class='column-4']")
for i in x:
    ind.append(i.text)    
s= driver.find_elements_by_xpath("//table[@id='tablepress-48']//td[@class='column-5']")
for i in s:
    sv.append(i.text)    
c= driver.find_elements_by_xpath("//table[@id='tablepress-48']//td[@class='column-6']")
for i in c:
    city.append(i.text)    
j= driver.find_elements_by_xpath("//table[@id='tablepress-48']//td[@class='column-7']")
for i in j:
    inv.append(i.text)    
t= driver.find_elements_by_xpath("//table[@id='tablepress-48']//td[@class='column-8']")
for i in t:
    invt.append(i.text)    
a= driver.find_elements_by_xpath("//table[@id='tablepress-48']//td[@class='column-9']")
for i in a:
    amount.append(i.text)
    
# scraping the August month data
d= driver.find_elements_by_xpath("//table[@id='tablepress-49']//td[@class='column-2']")
for i in d:
    dt.append(i.text)    
n= driver.find_elements_by_xpath("//table[@id='tablepress-49']//td[@class='column-3']")
for i in n:
    name.append(i.text)    
v= driver.find_elements_by_xpath("//table[@id='tablepress-49']//td[@class='column-4']")
for i in v:
    ind.append(i.text)    
s= driver.find_elements_by_xpath("//table[@id='tablepress-49']//td[@class='column-5']")
for i in s:
    sv.append(i.text)    
c= driver.find_elements_by_xpath("//table[@id='tablepress-49']//td[@class='column-6']")
for i in c:
    city.append(i.text)    
j= driver.find_elements_by_xpath("//table[@id='tablepress-49']//td[@class='column-7']")
for i in j:
    inv.append(i.text)    
t= driver.find_elements_by_xpath("//table[@id='tablepress-49']//td[@class='column-8']")
for i in t:
    invt.append(i.text)    
a= driver.find_elements_by_xpath("//table[@id='tablepress-49']//td[@class='column-9']")
for i in a:
    amount.append(i.text)
    
# scraping the September month data
d= driver.find_elements_by_xpath("//table[@id='tablepress-50']//td[@class='column-2']")
for i in d:
    dt.append(i.text)    
n= driver.find_elements_by_xpath("//table[@id='tablepress-50']//td[@class='column-3']")
for i in n:
    name.append(i.text)    
v= driver.find_elements_by_xpath("//table[@id='tablepress-50']//td[@class='column-4']")
for i in v:
    ind.append(i.text)    
s= driver.find_elements_by_xpath("//table[@id='tablepress-50']//td[@class='column-5']")
for i in s:
    sv.append(i.text)    
c= driver.find_elements_by_xpath("//table[@id='tablepress-50']//td[@class='column-6']")
for i in c:
    city.append(i.text)    
j= driver.find_elements_by_xpath("//table[@id='tablepress-50']//td[@class='column-7']")
for i in j:
    inv.append(i.text)    
t= driver.find_elements_by_xpath("//table[@id='tablepress-50']//td[@class='column-8']")
for i in t:
    invt.append(i.text)    
a= driver.find_elements_by_xpath("//table[@id='tablepress-50']//td[@class='column-9']")
for i in a:
    amount.append(i.text)    

# now creating a dataframe from all the collected information
data = list(zip(dt,name,ind,sv,city,inv,invt,amount))
df = pd.DataFrame(data, columns = ["Dates","Startup-name","Industry/Vertical","Sub-vertical","City",
                                   "Investor's Name","Investment Type","Amount"])
df

,Dates,Startup-name,Industry/Vertical,Sub-vertical,City,Investor's Name,Investment Type,Amount
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


**Q7)Write a program to scrap all the available details of best gaming laptops from digit.in.**

In [51]:
#connecting and giving url to the web driver 
driver = webdriver.Chrome("chromedriver.exe")
url = "https://www.digit.in/"
driver.get(url)
time.sleep(3)

# now we will search for best gaming laptops
best_gaming_lap = driver.find_element_by_xpath("//div[@class='listing_container']//ul//li[9]").click()
time.sleep(4)

# creating empty lists to store all gaming laptop information
lap_name = []
op_stm = []
display = []
processor = []
memory = []
weight = []
dimensions = []
graph_processor = []
prices = []

# fetching the laptop names
name = driver.find_elements_by_xpath("//table[@id='summtable']//tr//td[1]")
for i in name:
    lap_name.append(i.text)
    
# fetching information related to operating system
try:
    os = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[3]//td[3]")
    for i in os:
        op_stm.append(i.text)
except NoSuchElementException:
    pass

# fetching information for display
try:
    disp = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[4]//td[3]")
    for i in disp:
        display.append(i.text)
except NoSuchElementException:
    pass

# fetching information for processor
try:
    pro = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[5]//td[3]")
    for i in pro:
        processor.append(i.text)
except NoSuchElementException:
    pass

# fetching information for memory
try:
    mem = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[6]//td[3]")
    for i in mem:
        memory.append(i.text)
except NoSuchElementException:
    pass

# fetching information for weight
try:
    wgt = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[7]//td[3]")
    for i in wgt:
        weight.append(i.text)
except NoSuchElementException:
    pass

# fetching information for dimensions
try:
    dim = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[8]//td[3]")
    for i in dim:
        dimensions.append(i.text)
except NoSuchElementException:
    pass

# fetching information for graphic processor
try:
    g_pro = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[9]//td[3]")
    for i in g_pro:
        graph_processor.append(i.text)
except NoSuchElementException:
    pass

# fetching information for price
try:
    price = driver.find_elements_by_xpath("//td[@class='smprice']")
    for i in price:
        prices.append(i.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

# now creating a dataframe from all the collected details
data = list(zip(lap_name,op_stm,display,processor,memory,weight,dimensions,graph_processor,prices))
df = pd.DataFrame(data, columns = ["Laptop Name","Operating System","Display","Processor","Memory",
                                   "Weight(Kg)","Dimensions","Graphic-Processor","Price"])
df

,Laptop Name,Operating System,Display,Processor,Memory,Weight(Kg),Dimensions,Graphic-Processor,Price
0,Acer Nitro 5,Windows 10,"15.6"" (1920 x 1080)",AMD Ryzen 9 Octa Core | 2.4 GHz,1 TB HDD/16 GBGB DDR4,2.4,363.4 x 255 x 23.9,NVIDIA GeForce RTX 3070,"Rs 129,990"
1,MSI Stealth 15M,Windows 10,"15.6"" (1920 x 1080)",Intel Core i7 11th Gen - 11375H | NA,1 TB SSD/16 GBGB DDR4,1.7,358.3 x 248 x 16.15,NVIDIA GeForce RTX 3060,"Rs 134,990"
2,ASUS ROG Strix Scar 15,Windows 10,"15.6"" (2560 x 1440)",AMD Ryzen 9 Octa Core - 5900HX | 3.3 GHz,2 TB SSD/32 GBGB DDR4,2.30,354 x 259 x 22.6,NVIDIA GeForce RTX 3080,"Rs 268,990"
3,Alienware Area 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Gen Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,"Rs 342,989"
4,Alienware m15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Gen Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,"Rs 319,990"
5,ASUS ROG Strix Scar 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,"Rs 215,990"
6,ASUS ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Gen Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"Rs 164,990"
7,Lenovo Legion 5i,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Gen Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,"Rs 76,988"
8,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"Rs 185,000"
9,Acer Aspire 7 gaming laptop,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,"Rs 57,900"


**Q8)Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”**

In [52]:
#connecting and giving url to the web driver 
driver = webdriver.Chrome('chromedriver.exe')
url = 'https://www.forbes.com/'
driver.get(url)
time.sleep(3)

# clicking on the hamburger option on the left side of the page
optn = driver.find_element_by_xpath("//button[@class='icon--hamburger']")
optn.click()
time.sleep(1)

# now clicking on the 'Billionaires' option
bill = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]")
bill.click()
time.sleep(1)

# finally clicking on the 'World Billionaire' option after the mouse hover
world_bill = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")
world_bill.click()
time.sleep(1)

In [53]:
# fetching the rank
rank = []
try:
    rnk = driver.find_elements_by_xpath("//div[@class='rank']")
    for i in rnk:
        rank.append(i.text)
except NoSuchElementException:
    pass

# fetching the names of the billionaires
names = []
try:
    name = driver.find_elements_by_xpath("//div[@class='personName']")
    for i in name:
        names.append(i.text)
except NoSuchElementException:
    pass

# fetching the total net worth
net_worth = []
try:
    worth = driver.find_elements_by_xpath("//div[@class='netWorth']")
    for i in worth:
        net_worth.append(i.text)
except NoSuchElementException:
    pass

# fetching the age of the billionaires
Age = []
try:
    age = driver.find_elements_by_xpath("//div[@class='age']")
    for i in age:
        Age.append(i.text)
except NoSuchElementException:
    Age.append('-')
    
# fetching the citizenship of the billionaires
citizenship = []
try:
    cit = driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for i in cit:
        citizenship.append(i.text)
except NoSuchElementException:
    citizenship.append('-')
    
# fetching the source of incomes for the billionaires
source = []
try:
    src = driver.find_elements_by_xpath("//div[@class='source-column']")
    for i in src:
        source.append(i.text)
except NoSuchElementException:
    source.append('-')
    
# fetching the industry in which the billionaires are prominent
industry = []
try:
    ind = driver.find_elements_by_xpath("//div[@class='category']")
    for i in ind:
        industry.append(i.text)
except NoSuchElementException:
    industry.append('-')
    
# now creating a dataframe from all the collected information
data = list(zip(rank,names,net_worth,Age,citizenship,source,industry))
df = pd.DataFrame(data, columns = ["Rank", "Name", "Net worth", "Age", "Citizenship", "Source","Industry"])
df

,Rank,Name,Net worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
195,195.,Harry Triguboff,$11.2 B,88,Australia,real estate,Real Estate
196,197.,Leonid Fedun & family,$11.1 B,65,Russia,oil,Energy
197,197.,Eyal Ofer,$11.1 B,70,Israel,"real estate, shipping",Diversified
198,197.,Evan Spiegel,$11.1 B,30,United States,Snapchat,Technology


**Q9)Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted
from any YouTube Video.**

In [23]:
#connecting and giving url to the web driver 
driver=webdriver.Chrome('chromedriver.exe')
driver.get('https://www.youtube.com/')

In [26]:
#searching for the web element in the search box 
search_bar=driver.find_element_by_xpath('/html/body/ytd-app/div/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/form/div[1]/div[1]/input')
search_bar.send_keys("tajdar e haram")
time.sleep(5)

In [29]:
#searching and clicking on search button 
search_button=driver.find_element_by_xpath('//button[@class="style-scope ytd-searchbox"]')
search_button

<selenium.webdriver.remote.webelement.WebElement (session="e1678912314a739e9935a730c62e1cfd", element="82b6ac4e-cd2a-46b9-9ec7-4df1f63b7fbf")>

In [30]:
search_button.click()

In [31]:
#clicking on first video 
first_video=driver.find_element_by_xpath("/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer[1]/div[1]/div/div[1]/div/h3/a/yt-formatted-string")
first_video

<selenium.webdriver.remote.webelement.WebElement (session="e1678912314a739e9935a730c62e1cfd", element="b19500fc-f48f-490e-a1a9-d6efe8194b55")>

In [32]:
first_video.click()

In [38]:
# 1000 time we scroll down by 10000 in order to generate more Comments
for _ in range(1000):
    driver.execute_script("window.scrollBy(0,10000)")

In [42]:
#make empty lists
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

#scrape comments
cm = driver.find_elements_by_id("content-text")
for i in cm:
    if i.text is None:
        comments.append("--")
    else:
        comments.append(i.text)
        
time.sleep(5)


# scrape time when comment was posted
tm = driver.find_elements_by_xpath("//a[contains(text(),'ago')]")
for i in tm:
    Time.append(i.text)

for i in range(0,len(Time),2):
    comment_time.append(Time[i])
time.sleep(5)
    
# scrape the comment likes
like = driver.find_elements_by_xpath("//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for i in like:
    Likes.append(i.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])
    
    
time.sleep(2)
#creating dataframe
df=pd.DataFrame({'Comments':comments,
                'Time':comment_time,
                'Likes':No_of_Likes})
df

,Comments,Time,Likes
0,Tajalliyon ki ajab hai fizaa Madine mein\nNiga...,8 months ago,235
1,Let's talk about the beauty ornaments of his s...,6 days ago,245
2,I'm a Sikh. \nBut main addicted hun is naat ki...,3 weeks ago (edited),1.2K
3,Whenever I lost hope in any kind of situation....,1 day ago,14
4,The first Naat I wanted to listen to as 2022 b...,3 weeks ago (edited),659
5,Everytime I feel lost and sad I just listen to...,3 days ago,61
6,I m Christian. But I respect all religion's. ...,3 years ago,1K
7,"As this year 2021 cones to an end, listening t...",3 weeks ago,776
8,Saudações do Brasil para todos do Paquistão. ❤...,1 month ago,236
9,I am Hindu but I listen this song before 3year...,13 days ago,437


**Q10)10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews,
overall reviews, privates from price, dorms from price, facilities and property description.**

In [75]:
#connecting to the webdriver 
driver = webdriver.Chrome("chromedriver.exe")
url=('https://www.hostelworld.com/')
driver.get(url)

In [78]:
time.sleep(5)
#searching xpath on webpage
search_bar=driver.find_element_by_xpath('//input[@class="location-text"]')
#giving input in search bar
search_bar.send_keys("London, England")

In [80]:
#clicking on location 
locn=driver.find_element_by_xpath('//div[@class="label"]')
locn.click()

In [81]:
#searching data and pressing button
search_button=driver.find_element_by_id('search-button')
search_button.click()

In [93]:
# scraping the data for name of hostel
name_of_hostel=[]
name=driver.find_elements_by_xpath("//h2[@class='title title-6']")
for i in name:
    name_of_hostel.append(i.text)  
    
#scraping the data for distance of hostel from city center
distance_from_citycenter=[]
distance=driver.find_elements_by_xpath("//span[@class='description']")
for i in distance:
    distance_from_citycenter.append(i.text.replace('Hostel - ',''))
    
#scraping data for rating of the hostel 
total_rating=[]
rating=driver.find_elements_by_xpath("//div[@class='score orange big']")
for i in rating:
    total_rating.append(i.text)    
    
#scraping the data for review of hostel 
overall_review=[]
review1=driver.find_elements_by_xpath("//div[@class='keyword']")
for i in review1:
    overall_review.append(i.text)

    
#scraping data for overall review of hostel
reviews=[]
review=driver.find_elements_by_xpath("//div[@class='reviews']")
for i in review:
    reviews.append(i.text)  
    
#scraping the data for prices of rooms of hostel
private_price=[]
price=driver.find_elements_by_xpath("//div[@class='price title-5']")
for i in price:
    private_price.append(i.text)    
    
#scrpaing the data for private room price of hostel
privare_room_price=[]
for i in range(0,len(private_price),2):
    privare_room_price.append(private_price[i])
    
#scraping the data for dorms room price of hostel 
dorms_room_price=[]
for i in range(1,len(private_price),2):
    dorms_room_price.append(private_price[i].replace('\n',''))
    
#scraping the data for faciliteis offered by hostel
room_facilities=[]
facilty=driver.find_elements_by_xpath("//div[@class='facilities-label facilities']")
for i in facilty:
    room_facilities.append(i.text)   
    
#scraping the data for discription of the hostel
hostel_description=[]
description=driver.find_elements_by_xpath("//div[@class='rating-factors prop-card-tablet rating-factors small']")
for i in description:
    try:
        hostel_description.append(i.text.replace('\n',''))
        
    except:
         hostel_description.append('-')
            

In [92]:
#creating the DataFrame
hostels= pd.DataFrame({})
hostels['name']= name_of_hostel[:20]
hostels['distance from citycenter']= distance_from_citycenter[:20]
hostels['rating']= total_rating[:20]
hostels['overall review']= overall_review[:20]
hostels['privates from price']= privare_room_price[:20]
hostels['dorms from price']= dorms_room_price[:20]
hostels['facilities']= room_facilities[:20]
hostels['description']= hostel_description[:20]
hostels

,name,distance from citycenter,rating,overall review,privates from price,dorms from price,facilities,description
0,Mornington Camden,4.1km from city centre,8.3,Fabulous,Rs3277,Rs12769,Free WiFi,Perfect LocationSuperb StaffFantastic Cleanliness
1,Smart Russell Square Hostel,2.6km from city centre,6.7,Fabulous,Rs1205,Rs4984,Free WiFi\nFollows Covid-19 sanitation guidance,Perfect LocationBrilliant StaffAwesome Cleanli...
2,Smart Camden Inn Hostel,4.4km from city centre,8.5,Good,Rs1718,Rs12901,Free WiFi\nFollows Covid-19 sanitation guidance,Perfect LocationSuperb StaffFantastic Cleanliness
3,Selina Camden,5.5km from city centre,9.2,Fabulous,Rs4173.85 Rs3548,Rs4112,Free WiFi,Perfect LocationSuperb StaffFantastic Cleanliness
4,Queen Elizabeth Chelsea,5.7km from city centre,7.5,Superb,Rs1821,Rs6678,Free WiFi,Perfect LocationSuperb StaffAwesome Cleanliness
5,Pickwick Hall,2.3km from city centre,9.0,Very Good,Rs5533.52 Rs4980,Rs1673.24 Rs1506,Free Breakfast,Perfect LocationSuperb StaffFantastic Cleanliness
6,New Cross Inn Hostel,6.1km from city centre,7.4,Superb,Rs1108,Rs9713,Free WiFi\nFree Breakfast,Wonderful LocationMarvellous StaffAwesome Clea...
7,No.8 Willesden Hostel London,10km from city centre,7.6,Very Good,Rs2125,Rs6206,Free WiFi\nFollows Covid-19 sanitation guidance,Wonderful LocationSuperb StaffAwesome Cleanliness
8,Safestay London Elephant & Castle,1.7km from city centre,8.0,Very Good,Rs2293,Rs5565,Free WiFi\nFollows Covid-19 sanitation guidance,Perfect LocationMarvellous StaffExcellent Clea...
9,Barmy Badger Backpackers,5.5km from city centre,9.3,Fabulous,Rs855,Rs6418,Free WiFi\nFree Breakfast,Perfect LocationSuperb StaffFantastic Cleanliness
